In [1]:
#!pip install --upgrade pip
#!pip install transformers==3.1.0

In [1]:
from transformers import pipeline
from os import listdir
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np

I0802 19:35:29.180007 139830866601792 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
I0802 19:35:29.203221 139830866601792 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [6]:
classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU


In [247]:
#path_screenplays_scenes='./transcripts/shot-aligned_transcripts_2021_Max_Jack_Tanya.csv'
path_screenplays_scenes='./transcripts/shot-aligned_transcripts_2021_Peggy_Archie.csv'





In [233]:


def get_results(df, classifier, topic_labels):
    df['topic'] = None
    df['score'] = None
    
    for i in tqdm(df.index):
        text = df.loc[i, 'transcript']
        if text is not None:
            result = classifier(text, topic_labels)
            #df.loc[i, 'topic'] = str(str(result['labels'][0])+"+"+str(result['labels'][1]))
            #df.loc[i, 'score'] = str(str(result['scores'][0])+"+"+str(result['scores'][1]) 
            df.loc[i, 'topic'] = result['labels'][0]
            df.loc[i, 'score'] = result['scores'][0]
            #if result['labels'][0]=='surprise':
                #df.loc[i, 'score_hf'] = result['scores'][0]
            #else:
                #df.loc[i, 'score_hf'] = -1
            
    return df


    
    
    

    




In [2]:
#Initial experiments with other candidate labels
"""candidate_labels_df=pd.read_csv('life_events_scale.csv',sep='\t', names=['topic','weight'])
candidate_labels_df=candidate_labels_df


candidate_labels=dict(zip(candidate_labels_df.topic, candidate_labels_df.weight))
for topic,weight in tqdm(candidate_labels.items()):
    print(topic)
    print(weight)"""
    


In [223]:
soap_opera_scale={'extramarital affair': 1/1.98, 'get divorced': 1/1.96,'illegitimate child': 1/1.45,'institutionalized for emotional problem': 1/1.43,'happily married': 1/4.05,'serious accident': 1/2.96,'murdered': 1/1.81,'attempt suicide': 1/1.26,'blackmailed': 1/1.86,'unfaithful spouse': 1/2.23,'sexually assaulted': 1/2.60,'abortion': 1/1.41}

In [249]:
def get_results_several_binary(df, classifier, topic_labels):
    for topic,weight in tqdm(topic_labels.items()):
        df[topic] = None
        for i in df.index:
            text = df.loc[i, 'transcript']
            if text is not None:
                result = classifier(text,topic,multi_class=False)
                df.loc[i, topic] = result['scores'][0]*weight
        #df.to_csv('soap_opera_Peggy_Archie.csv')
        

          
    return df

In [4]:

first_results=get_results_several_binary(df_with_text,classifier,candidate_labels)
cols_to_keep=['Unnamed: 0', 'begin', 'end', 'filename', 'shot_id', 'transcript']
first_results['score'] = first_results.drop(cols_to_keep, axis=1).max(axis=1)
first_results=first_results.sort_values(by=['score'],ascending=False)
first_results.to_csv('soap_opera_Peggy_Archie.csv')

    
    
